<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in the city of Toronto, Canada</font></h1>

In this assignment, I will try to explore, segment, and cluster the neighborhoods in the city of Toronto. 

For the Toronto neighborhood data,a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. We are going to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format.

#### Installing Required Packages

In [3]:
!conda install beautifulsoup4
!conda install lxml
!conda install requests
print('Packages installed successfully')

Fetching package metadata ...........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
beautifulsoup4            4.6.3                    py35_0  
Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    libgcc-ng: 7.2.0-h7cc24e2_2     --> 8.2.0-hdf63c60_1    
    libxml2:   2.9.4-h6b072ca_5     --> 2.9.8-hf84eae3_0    
    libxslt:   1.1.29-hcf9102b_5    --> 1.1.32-h1312cb7_0   
    lxml:      4.1.0-py35ha401a81_0 --> 4.2.5-py35hefd8a0e_0

libgcc-ng-8.2. 100% |################################| Time: 0:00:00  85.73 MB/s
libxml2-2.9.8- 100% |################################| Time: 0:00:00  65.22 MB/s
libxslt-1.1.32 100% |################################| Time: 0:00:00  64.13 MB/s
lxml-4.2.5-py3 100% |################################| Time: 0:00:00  14.48 MB/s
Fet

#### Importing all the required libraries

In [7]:
from bs4 import BeautifulSoup
import requests

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00   1.08 MB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00   1.68 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.0-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   2.51 MB/s
branca-0.3.0-p 100% |################################| Time: 0:00:00  25.59 MB/s
vincent-0.4.4- 100% |###################

#### Using Requests.get() to get the required web page and scrape the webpage using BeautifulSoup package

In [12]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')


#### Converting the table to a Pandas Dataframe

In [11]:
table = soup.find('table')
table_rows = table.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    l.append(row)
df = pd.DataFrame(l, columns=["Postcode", "Borough", "Neighbourhood"])
df.head()

,Postcode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [22]:
df1=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index() 
df1.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Rouge,Malvern"
2,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
3,M1E,Scarborough,"Guildwood,Morningside,West Hill"
4,M1G,Scarborough,Woburn


#### Dropping rows where Borough has a value of 'Not assigned'

In [59]:
df2 =df1.drop(df1[df1.Borough == 'Not assigned'].index).reset_index()
del df2['index']
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### If a cell has a borough but a Not assigned neighborhood, then making the neighborhood value to be the same as the borough. 

In [60]:
df2.loc[df2['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df2['Borough']
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [64]:
#Checking
df_chk = df2.loc[df2['Postcode']=="M7A"]
df_chk

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


#### Print the number of rows of the dataframe

In [65]:
df2.shape

(103, 3)